# Ports

> TODO fill in description

In [1]:
#| default_exp ports

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| hide
import nbdev; nbdev.nbdev_export()

In [4]:
#|export
import asyncio
from abc import ABC, abstractmethod
from typing import Type, Callable, Any
import inspect
from collections import deque

import fbdev
from fbdev.packet import Packet

In [5]:
#|export
__all__ = [
    'BasePort',
    'BaseInputPort', 'SingleInputPort', 'DequeInputPort',
    'BaseOutputPort', 'SingleOutputPort', 'DequeOutputPort'
]

In [ ]:
#|export
class BasePort(ABC):
    def __init__(self,
                 name:str,
                 dtype:type,
                 data_validator:Callable[[Any], bool]):
        self.name = name
        self.idx = None
        self._component_process = None
        
        self.dtype = dtype
        self.data_validator = data_validator
        
        self._packet = None
        self._port_filled_event = asyncio.Event()
        self._port_emptied_event = asyncio.Event()
        
    def attach(self, component_process, port_index:int):
        if self._component_process is not None:
            #TODO proper exception in BasePort.attach
            raise Exception("Port is already attached.")
        self._component_process = component_process
        self.idx = port_index
        
    def is_filled(self):
        return self._packet is not None
        
    def _check_packet_data_dtype(self,
                                 packet_data):
        if self.dtype is not None:
            if type(packet_data) != self.dtype:
                #TODO logging in BasePort._check_packet_data_dtype
                #TODO proper exceptions in BasePort._check_packet_data_dtype
                #TODO unit test BasePort._check_packet_data_dtype
                raise Exception("Wrong dtype.")
        
    def _check_packet_dtype(self,
                            packet: Type[Packet]):
        if self.dtype is not None:
            if packet.dtype != self.dtype:
                #TODO logging in BasePort._check_packet_dtype
                #TODO proper exceptions in BasePort._check_packet_dtype
                #TODO unit test BasePort._check_packet_dtype
                raise Exception("Wrong dtype.")
            
    def _validate_packet_data(self,
                       packet_data):
        if self.data_validator is not None:
            if not self.data_validator(packet_data):
                #TODO logging in BasePort._validate_packet_data
                #TODO proper exceptions in BasePort._validate_packet_data
                #TODO unit test BasePort._validate_packet_data
                raise Exception("Data is invalid.")

# Input Ports

In [ ]:
#|export
class BaseInputPort(BasePort):
    def __init__(self,
                 name:str,
                 dtype: type=None,
                 data_validator: Callable[[Any], bool]=None):
        super.__init__(name, dtype, data_validator)
        
    @abstractmethod
    async def load(self,
            packet: Packet):
        pass
        
    @abstractmethod
    async def get(self):
        pass

In [ ]:
#|export
class SingleInputPort(BaseInputPort):
    def __init__(self,
                 name:str,
                 dtype:type=None,
                 data_validator:Callable[[Any], bool]=None):
        super.__init__(name, dtype, data_validator)
        
    async def load(self,
            packet: Packet):
        
        self._check_packet_dtype(packet)
            
        await self._port_emptied_event.wait()
        self._port_emptied_event.clear()
            
        self._packet = packet
        self._port_filled_event.set()
        
    async def get(self):
        await self._port_filled_event.wait()
        self._port_filled_event.clear()
        
        packet_data = self._packet.get_data()
        if inspect.isawaitable(packet_data): packet_data = await packet_data
        self._validate_packet_data(packet_data)
        self._packet = None
        
        self._port_emptied_event.set()
        
        return packet_data

In [ ]:
#|export
class DequeInputPort(BaseInputPort):
    def __init__(self,
                 name:str,
                 maxsize:int,
                 dtype:type=None,
                 data_validator:Callable[[Any], bool]=None,
                 ):
        super.__init__(name, dtype, data_validator)
        self.maxsize = maxsize
        del self._packet
        self._ports = [SingleInputPort(dtype, data_validator) for i in range(self.size)]
        self._filled_ports = deque(maxlen=self.size)
        self._empty_ports = []
        
    async def load(self,
            packet: Packet):
        
        if len(self._empty_ports) == 0:
            await self._port_emptied_event.wait()
            self._port_emptied_event.clear()

        port = self._empty_ports.pop()
        await port.load(packet)
        self._filled_ports.append(port)
        
        self._port_filled_event.set()
        
    async def loadleft(self,
            packet: Packet):
        
        if len(self._empty_ports) == 0:
            await self._port_emptied_event.wait()
            self._port_emptied_event.clear()

        port = self._empty_ports.pop()
        await port.load(packet)
        self._filled_ports.appendleft(port)
        
        self._port_filled_event.set()
        
    async def getall(self):
        if len(self._filled_ports) == 0: await self._port_filled_event.wait()
            
        packets = [await port.get() for port in self._filled_ports]
        self._empty_ports += list(self._filled_ports)
        self._filled_ports.clear()
        
        self._port_filled_event.clear()
        self._port_emptied_event.set()
        
        return packets
        
    async def get(self):
        if len(self._filled_ports) == 0: await self._port_filled_event.wait()
            
        port = self._filled_ports.popleft()
        packet = await port.get()
        self._empty_ports.append(port)
            
        if len(self._filled_ports) == 0: self._port_filled_event.clear()
            
        self._port_emptied_event.set()
            
        return packet
    
    async def getright(self):
        if len(self._filled_ports) == 0: await self._port_filled_event.wait()
            
        port = self._filled_ports.pop()
        packet = await port.get()
        self._empty_ports.append(port)
            
        if len(self._filled_ports) == 0: self._port_filled_event.clear()
            
        self._port_emptied_event.set()
            
        return packet

# Output Ports

In [ ]:
#|export
class BaseOutputPort(BasePort):
    def __init__(self,
                 name:str,
                 dtype:type=None,
                 data_validator:Callable[[Any], bool]=None):
        super.__init__(name, dtype, data_validator)
        
    @abstractmethod
    async def put(self,
            packet_data):
        pass
        
    @abstractmethod
    async def unload(self):
        pass

In [ ]:
#|export
class SingleOutputPort(BaseOutputPort):
    def __init__(self,
                 name:str,
                 dtype:type=None,
                 data_validator:Callable[[Any], bool]=None):
        super.__init__(name, dtype, data_validator)
        
    async def put(self,
            packet_data):
        self._check_packet_data_dtype(packet_data)
        self._validate_packet_data(packet_data)
        
        await self._port_emptied_event.wait()
        self._port_emptied_event.clear()
        self._packet = Packet(packet_data)
        
        self._port_filled_event.set()
        
    async def unload(self):
        await self._port_filled_event.wait()
        self._port_filled_event.clear()
        
        packet = self._packet
        self._packet = None
        self._port_emptied_event.set()
        return packet

In [ ]:
#|export
class DequeOutputPort(BaseOutputPort):
    def __init__(self,
                 maxsize:int,
                 name:str,
                 dtype:type=None,
                 data_validator:Callable[[Any], bool]=None,
                 ):
        super.__init__(name, dtype, data_validator)
        self.maxsize = maxsize
        del self._packet
        self._ports = [SingleOutputPort(dtype, data_validator) for i in range(self.size)]
        self._filled_ports = deque(maxlen=self.size)
        self._empty_ports = []
        
    async def put(self,
            packet_data):
        
        if len(self._empty_ports) == 0:
            await self._port_emptied_event.wait()
            self._port_emptied_event.clear()

        port = self._empty_ports.pop()
        await port.put(packet_data)
        self._filled_ports.append(port)
        
        self._port_filled_event.set()
        
    async def putleft(self,
            packet_data):
        
        if len(self._empty_ports) == 0:
            await self._port_emptied_event.wait()
            self._port_emptied_event.clear()

        port = self._empty_ports.pop()
        await port.put(packet_data)
        self._filled_ports.appendleft(port)
        
        self._port_filled_event.set()
        
    async def unload(self):
        if len(self._filled_ports) == 0: await self._port_filled_event.wait()
            
        port = self._filled_ports.popleft()
        packet = await port.unload()
        self._empty_ports.append(port)
            
        if len(self._filled_ports) == 0: self._port_filled_event.clear()
            
        self._port_emptied_event.set()
            
        return packet